In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50
## Install xlrd package to load Excel files
# conda install openpyxl
## conda install xlrd

In [2]:
RAND_STATE = 34 # for reproducible shuffling
TT_RATIO = 0.3 # test/train

In [3]:
from scipy.stats import iqr
def remove_outliers(df):
    for c in df.columns:
            pct_75 = np.percentile(df[c], 75)
            pct_25 = np.percentile(df[c], 25)
            upper_bound = pct_75 + 1.5*iqr(df[c])
            lower_bound = pct_25 - 1.5*iqr(df[c])
            condition = (df[c] < upper_bound) & (df[c] > lower_bound)
            df[c] = df[c][condition]  # Filter out the outliers
    return df

In [4]:
hk_df= pd.read_csv('/Airbnb_raw.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Airbnb_raw.csv'

In [ ]:
# Removing NaNs 
na_idcs = hk_df[hk_df.isna().any(axis=1)].index
hk_df = pd.DataFrame(hk_df).drop(na_idcs)
hk_df


In [ ]:
hk_df.info()

In [ ]:
#Checking coorelation between superhost and costumer satisfaction 
corr = hk_df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True # trick to filter out the upper-right triangle, which is redundant due to symmetry
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(16, 14))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)

In [ ]:
def drop_columns(hk_df) :
    hk_df.drop(columns=["Attraction Index","Restraunt Index"], inplace=True)
    return hk_df

In [ ]:
 hk_df1=drop_columns(hk_df)

In [ ]:
corr = hk_df1.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True # trick to filter out the upper-right triangle, which is redundant due to symmetry
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(16, 14))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)

In [ ]:
# function to change bool into float
def boolstr_to_floatstr(v):
    if v == 'True':
        return '1'
    elif v == 'False':
        return '0'
    else:
        return v

In [ ]:
hk_df1['Shared Room'] = np.vectorize(boolstr_to_floatstr)(hk_df1['Shared Room']).astype(float)
hk_df1['Private Room'] = np.vectorize(boolstr_to_floatstr)(hk_df1['Private Room']).astype(float)
hk_df1['Multiple Rooms'] = np.vectorize(boolstr_to_floatstr)(hk_df1['Multiple Rooms']).astype(float)
hk_df1['Superhost'] = np.vectorize(boolstr_to_floatstr)(hk_df1['Superhost']).astype(float)
hk_df1['Bedrooms'] = np.vectorize(boolstr_to_floatstr)(hk_df1['Bedrooms']).astype(float)
hk_df1['Business'] = np.vectorize(boolstr_to_floatstr)(hk_df1['Business']).astype(float)

In [ ]:
numericalX = hk_df1.select_dtypes(np.number)
categoricalX = hk_df1.select_dtypes(object)

In [ ]:
numericalX.info()

In [ ]:
categoricalX['Bedrooms'] = numericalX['Bedrooms']
numericalX = numericalX.drop(columns=['Bedrooms'])

In [ ]:
categoricalX.info()

In [ ]:
sns.pairplot(numericalX.select_dtypes(np.number))